# Team 5 💣
1. Maximus Lee
2. Aloysius Woo
3. Lim Huai Fu
4. Tan Ai Xin
5. Jin Zhenglong

## Setup

In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

### Install Environment
Est Time: 8 mins



In [ ]:
# @title Python

%cd /content
!sudo apt-get install python3.8
!sudo apt-get install python3.8-distutils

!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip
!apt-get install libcairo2-dev
!apt update

In [ ]:
#@title GitHub Repos

# Empty default folder
!find /content -mindepth 1 -delete

# nice try
!git clone https://github.com/maximus-lee-678/ict3104_team_05.git .

In [ ]:
#@title FollowYourPose/MMPose Folder Scaffolding

# FollowYourPose
%cd /content/FollowYourPose
%mkdir /content/FollowYourPose/checkpoints

# MMPose
%cd /content/MMPose
%mkdir video_input
%mkdir video_output

In [ ]:
#@title FollowYourPose/MMPose Libaries
# https://github.com/open-mmlab/mmpose
# https://blog.csdn.net/qq_21532607/article/details/130226728
# https://colab.research.google.com/github/open-mmlab/mmpose/blob/master/demo/MMPose_Tutorial.ipynb

%cd /content
!python3.8 -m pip install -r other_files/requirements/requirements_colab.txt

!python3.8 -m mim install mmengine
!python3.8 -m mim install mmpose
!python3.8 -m mim install "mmcv>=2.0.0"
!python3.8 -m mim install "mmdet>=3.0.0"

In [ ]:
#@title Others

!pip install moviepy
!apt-get install imagemagick
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

### Mount Drive 🗻

1. Run the first cell to authenicate
2. Click on the URL of the Failure in the output Failure("Error opening URL: ...")
3. Select the ICT3104 shared account
4. Once successful, you can close the tab
5. Run the next cell
6. Done

In [ ]:
# @title First Cell
%cd -q /content
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
# @title Next Cell
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd -q /content
!mkdir gdrive
%cd -q gdrive
!mkdir MyDrive
%cd -q ..
%cd -q ..
!google-drive-ocamlfuse /content/gdrive/MyDrive

### Symbolic Linking

In [ ]:
# Root Directory
ROOT_DIR_PATH = "/content"

# Video
VIDEO_DIR_PATH = f"{ROOT_DIR_PATH}/video"
%mkdir {VIDEO_DIR_PATH}
VIDEO_SKELETON_DIR_PATH = f"{ROOT_DIR_PATH}/video/Skeleton"
%mkdir {VIDEO_SKELETON_DIR_PATH}

# I/O Files
TRAINING_CONTENT_DIR_PATH = f"{ROOT_DIR_PATH}/training_content"
%mkdir {TRAINING_CONTENT_DIR_PATH}
CUSTOM_MODEL_DIR_PATH = f"{ROOT_DIR_PATH}/custom_model"
%mkdir {CUSTOM_MODEL_DIR_PATH}
INFERENCE_OUTPUT_DIR_PATH = f"{ROOT_DIR_PATH}/inference_output"
%mkdir {INFERENCE_OUTPUT_DIR_PATH}

MMPOSE_DIR_PATH = f"{ROOT_DIR_PATH}/MMPose"
FYP_DIR_PATH = f"{ROOT_DIR_PATH}/FollowYourPose"

# FYP Config
CONFIG_DIR_PATH = f"{FYP_DIR_PATH}/configs"
%mkdir {CONFIG_DIR_PATH}

In [ ]:
# symbolic link edition
!ln -s /content/gdrive/MyDrive/ICT3104/video/* /content/video

# copy edition
# %cp -r /content/gdrive/MyDrive/ICT3104/video/* /content/video/

In [ ]:
# Checkpoints folder

# symbolic link edition
!ln -s /content/gdrive/MyDrive/ICT3104/checkpoints/* /content/FollowYourPose/checkpoints

# copy edition
# %cp -r /content/gdrive/MyDrive/ICT3104/checkpoints/* /content/FollowYourPose/checkpoints/

In [ ]:
# Charade Lookup Files
%mkdir /content/charades_lookup/

# symbolic link edition
!ln -s /content/gdrive/MyDrive/ICT3104/charades_lookup/* /content/charades_lookup

### Pre-setup

In [ ]:
import sys

# Check if the path is not already in sys.path before appending
path_to_append = '/usr/local/lib/python3.8/dist-packages'
if path_to_append not in sys.path:
    sys.path.append(path_to_append)

import os
import csv
import datetime
import time
import subprocess

from omegaconf import OmegaConf
import yaml

from ipywidgets import Dropdown, Output, Layout, widgets, Button, VBox, HBox
from IPython.display import display, Markdown, HTML, Video, clear_output

from moviepy.editor import VideoFileClip, clips_array, TextClip, CompositeVideoClip, ColorClip
from moviepy.config import change_settings
from PIL import Image, ImageDraw, ImageFont, ImageSequence

change_settings({"IMAGEMAGICK_BINARY": "/usr/bin/convert"})

### Utils

In [ ]:
#@title Configuration Presets

# Expected Configurations of a Inference yaml
expected_inference_config = {
  "pretrained_model_path": str,
  "output_dir": str,
  "validation_data": {
      "prompts": list,
      "video_length": int,
      "width": int,
      "height": int,
      "num_inference_steps": int,
      "guidance_scale": float,
      "use_inv_latent": bool,
      "num_inv_steps": int,
      "dataset_set": str
  },
  "train_batch_size": int,
  "validation_steps": int,
  "resume_from_checkpoint": str,
  "seed": int,
  "mixed_precision": str,
  "gradient_checkpointing": bool,
  "enable_xformers_memory_efficient_attention": bool
}

# Expected Configurations of a Training yaml
expected_training_config = {
  "pretrained_model_path": str,
  "output_dir": str,
  "train_data": {
    "video_path": str,
    "n_sample_frames": int,
    "width": int,
    "sample_frame_rate": int
  },
  "learning_rate": float,
  "train_batch_size": int,
  "max_train_steps": int,
  "trainable_modules": list,
  "seed": int,
  "mixed_precision": str,
  "use_8bit_adam": bool,
  "gradient_checkpointing": bool,
  "enable_xformers_memory_efficient_attention": bool
}

# Define the options for boolean dropdown
boolean_dropdown = [True, False]

# Create layout
configs_config_style = {'description_width': '150px'}
configs_config_layout = widgets.Layout(width="500px")
configs_config_button_layout = widgets.Layout(margin='0px 0px 20px 354px')

In [ ]:
#@title Configuration Functions

# Find all models and creates a list of the models' path in the directory
def create_model_list():
  model_options = [("Default", f"{FYP_DIR_PATH}/checkpoints/stable-diffusion-v1-4")]
  for model_name in os.listdir(CUSTOM_MODEL_DIR_PATH):
    model_path = os.path.join(CUSTOM_MODEL_DIR_PATH, model_name)
    if os.path.isdir(model_path):
      model_options.append((model_name, f"{CUSTOM_MODEL_DIR_PATH}/{model_name}"))
  return model_options

def create_video_path_list():
  video_path_options = [("None", "/content/training_content/")]
  for video_name in os.listdir(TRAINING_CONTENT_DIR_PATH):
    video_path = os.path.join(TRAINING_CONTENT_DIR_PATH, video_name)
    if os.path.isdir(video_path):
      video_path_options.append((video_name, f"{TRAINING_CONTENT_DIR_PATH}/{video_name}"))
  return video_path_options

# Check if the yaml is in the correct structure for inference
def compare_dict_structure(expected, yaml_config):
  for key, value in expected.items():
    if not isinstance(value, dict):
      if key not in yaml_config or not isinstance(yaml_config[key], value): # Check if key exists and if data type matches
        if key in yaml_config and isinstance(list(yaml_config[key]), value): # Special check for list because omegaconf sees it as a ListConfig
          return True
        return False
    elif key in yaml_config: # If value is a dict recurse to its first non dict value
      compare_dict_structure(value, yaml_config[key])
  return True

In [ ]:
#@title Post-Process Inference Functions

def postprocess_gif(inf_path, prompt, size):

  pose_path = f'{inf_path}/pose.gif'
  gif_video_path = f'{inf_path}/raw/{prompt}.gif'
  gif_output_path = f"{inf_path}/processed/{prompt}.gif"

  # Load video
  skeleton_video = VideoFileClip(pose_path)
  gif_video = VideoFileClip(gif_video_path)

  # Video editting
  skeleton_video = skeleton_video.resize((size, size))
  gif_video = gif_video.resize((size, size))

  # Combine videos side by side
  result = clips_array([[skeleton_video, gif_video]])

  # Add captions
  txt_clip = TextClip(prompt, font="Amiri-bold", fontsize=30, color='white')
  txt_clip = txt_clip.set_duration(result.duration)
  txt_clip = txt_clip.set_position(("center", "bottom"))

  # Create a black bar for captions
  black_clip = ColorClip(size=(result.w, result.h + 40), color=(0, 0, 0), duration=result.duration)

  # Combine clips with captions
  gif_with_text = CompositeVideoClip([black_clip, txt_clip, result])

  # Write the final GIF
  gif_with_text.write_gif(gif_output_path, fps=10)

def postprocess_mmpose(skeleton_path, video_length, output_dir):

  # Get the video duration using ffprobe
  duration = float(subprocess.check_output(['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', skeleton_path]))

  # Set your desired duration
  desired_duration = video_length * 130 / 1000

  # Calculate the speedup factor
  speedup_factor = desired_duration / duration

  # Run the FFmpeg command to adjust the video's duration
  output_path = f'{output_dir}/pose.gif'
  !ffmpeg -i $skeleton_path -vf "setpts=$speedup_factor*PTS,fps=10" $output_path

## Video Browsing

In [ ]:
# @title Video Selection { display-mode: "form" }

# Helper Functions
def getFolderContent(folder_name):
  subfolders = []
  for content in os.listdir(VIDEO_DIR_PATH):
        content_path = os.path.join(VIDEO_DIR_PATH, content)

        # Check if it's a directory and not hidden
        if os.path.isdir(content_path) and not content.startswith("."):
            subfolders.append(content)

  return subfolders

# Init
video_dir_folders = os.listdir(VIDEO_DIR_PATH)
video_dir_folder_content = getFolderContent(video_dir_folders)

# Create layout
layout_single_button = widgets.Layout(width='212px',margin='0px 0px 20px 90px')
layout_double_button = widgets.Layout(width='104px')
layout_hbox = widgets.Layout(margin='0px 0px 0px 88px')
layout_output = widgets.Layout(margin='0px 0px 20px 0px', display='flex', align_items='flex-start')

# Create widgets
video_output_placeholder = widgets.Output(layout=layout_output)
video_subdir_dropdown = widgets.Dropdown(options=video_dir_folder_content, description='Folder:', value=None, disabled=False)
video_dropdown = widgets.Dropdown(options=[], description='Video:', disabled=True)
video_display_button = widgets.Button(description="Display", disabled=True, layout=layout_double_button)
video_refresh_button = widgets.Button(description="Refresh", disabled=False, layout=layout_double_button)
video_mmpose_button = widgets.Button(description="MMPose", disabled=True, layout=layout_double_button)
video_fyp_button = widgets.Button(description="FYP", disabled=True, layout=layout_double_button)

video_hbox_1 = widgets.HBox([video_display_button, video_refresh_button], layout=layout_hbox)
video_hbox_2 = widgets.HBox([video_mmpose_button, video_fyp_button], layout=layout_hbox)

video_output_placeholder_content = HTML("""
  <div style="width: 512px; height: 512px; border-radius: 5%; background-color: black; margin: 0 auto; display: flex; justify-content: center; align-items: center;">
      <div style="width: 500px; height: 500px; border-radius: 5%; border: 2px solid white;" />
  </div>
""")

# Create listeners
## Update video dropdown options based on the selected folder
def video_subdir_select(change):
    selected_video_folder = video_subdir_dropdown.value

    if selected_video_folder != None:
      selected_VIDEO_DIR_PATH = f"{VIDEO_DIR_PATH}/{selected_video_folder}"
      selected_video_dir_content = [file for file in os.listdir(selected_VIDEO_DIR_PATH) if file.endswith('.mp4')]
    else:
      selected_video_dir_content = []

    video_dropdown.options = selected_video_dir_content
    if not selected_video_dir_content:
        video_dropdown.disabled = False
        video_dropdown.value = None
    else:
        video_dropdown.disabled = False

## Display the selected video
def display_selected_video(change):
    selected_video = video_dropdown.value
    selected_video_folder = video_subdir_dropdown.value

    if selected_video:
        video_path = f"{VIDEO_DIR_PATH}/{selected_video_folder}/{selected_video}"
        video_display = Video(video_path, width=512, height=512, embed=True)

        # Clear the output placeholder and display the video
        with video_output_placeholder:
            clear_output()
            display(video_display)

## Refresh folder and directory
def refresh_folder_and_directory(change):
    video_dir_folders = os.listdir(VIDEO_DIR_PATH)
    video_dir_folder_content = getFolderContent(video_dir_folders)

    video_dropdown.options = []
    video_dropdown.value = None

    video_subdir_dropdown.options = video_dir_folder_content
    video_subdir_dropdown.value = None



## Enable button when a valid video is picked
def enable_button(change):
    if video_dropdown.value:
        video_display_button.disabled = False
        video_mmpose_button.disabled = False
        video_fyp_button.disabled = False
    else:
        video_display_button.disabled = True
        video_mmpose_button.disabled = True
        video_fyp_button.disabled = True

# Attach Listeners
video_subdir_dropdown.observe(video_subdir_select, 'value')
video_display_button.on_click(display_selected_video)
video_refresh_button.on_click(refresh_folder_and_directory)
video_dropdown.observe(enable_button, 'value')


# Display fields
with video_output_placeholder:
  display(video_output_placeholder_content)
display(video_output_placeholder)
display(video_subdir_dropdown)
display(video_dropdown)

display(video_hbox_1)
display(video_hbox_2)


## Inference: MMPOSE

In [ ]:
# @title Run MMPose Inference { display-mode: "form" }

# Helper Functions
## Re-encode video due to H.264 video encoding error
def reencode_video(input_file):
  temp_output_file = f"{VIDEO_SKELETON_DIR_PATH}/output.mp4"
  !ffmpeg -i {input_file} -c:v libx264 -crf 23 -c:a aac -strict experimental {temp_output_file}

  os.remove(input_file)
  os.rename(temp_output_file, input_file)

# Create layout
layout_single_long_button = widgets.Layout(margin='0px 0px 20px 154px')

# Create widgets
selected_mmpose_video_input = widgets.Text(placeholder='Select a video above', description="Video:", disabled=True)
inf_mmpose_button = widgets.Button(description="Start Inference", disabled=True, layout=layout_single_long_button)

# Create listeners
## Retrieve video input
def update_mmpose_input_video(change):
    selected_video = video_dropdown.value
    selected_mmpose_video_input.value = selected_video
    if selected_video:
        inf_mmpose_button.disabled = False
    else:
        inf_mmpose_button.disabled = True

## Run mmpose
def run_mmpose_inference(button):
    selected_video = video_dropdown.value
    video_path = f"/content/video/{video_subdir_dropdown.value}/{selected_video}"

    %cd -q {MMPOSE_DIR_PATH}

    if not os.path.exists(VIDEO_SKELETON_DIR_PATH):
        os.mkdir(VIDEO_SKELETON_DIR_PATH)

    # Start inference
    !python demo/inferencer_demo.py \
        {video_path}  \
        --pose2d human --vis-out-dir {VIDEO_SKELETON_DIR_PATH} \
        --black-background

    input_file = f"{VIDEO_SKELETON_DIR_PATH}/{selected_video}"
    reencode_video(input_file)

    clear_output()
    mmpose_inf_display()
    print(f"Done [Output: {input_file}]")

# Attach Listeners
video_mmpose_button.on_click(update_mmpose_input_video)
inf_mmpose_button.on_click(run_mmpose_inference)

# Display fields
def mmpose_inf_display():
  display(selected_mmpose_video_input)
  display(inf_mmpose_button)
mmpose_inf_display()

## Inference: FYP

In [ ]:
# @title Inference Configuration { display-mode: "form" }

# Init
%cd -q {FYP_DIR_PATH}

# Initialize load_config with a default value
load_config = None

# Define a container for displayed widgets
displayed_widgets = []

# Get a list of all files in the directory
config_files = [f for f in os.listdir(CONFIG_DIR_PATH) if os.path.isfile(os.path.join(CONFIG_DIR_PATH, f))]

# Create a dropdown widget with the list of config files
config_files_dropdown = Dropdown(
  options=["- Select an Item -"] + config_files,
  description='Select a Config File:',
  layout=Layout(width="500px"),
  style={'description_width': '150px'}
)

# Function to clear displayed widgets (excluding the dropdown)
def clear_displayed_widgets():
    for widget in displayed_widgets:
        widget.close()
    displayed_widgets.clear()
    display(config_files_dropdown)  # Display the dropdown again

# Function to update the load_config variable based on the selected filename
def update_load_config(change):
    global load_config
    selected_filename = change.new
    if selected_filename and selected_filename != "- Select an Item -":
        clear_output(wait=True)  # Clear the output area
        clear_displayed_widgets()  # Clear previously displayed widgets

        # Load yaml file
        sample_yaml_path = f"{CONFIG_DIR_PATH}/{selected_filename}"
        load_config = OmegaConf.load(sample_yaml_path)
        print(f"Editing config from: {sample_yaml_path}")

        # Check if the yaml configuration matches expected training config
        if compare_dict_structure(expected_inference_config, load_config):

          # Create a list of model names and paths in the directory
          model_options = create_model_list()

          # =====================================
          ## Basic Data
          config_subheader1 = widgets.HTML(value="<h3>Basic Data</h3>")
          config_pretrained_model_path = Dropdown(options=model_options, description="pretrained_model_path:", value=load_config.pretrained_model_path, style=configs_config_style, layout=configs_config_layout)
          config_output_dir_name = widgets.Text(description="output_dir_name:", value="", style=configs_config_style, layout=configs_config_layout)
          config_train_batch_size = widgets.IntText(description="train_batch_size:", value=load_config.train_batch_size, style=configs_config_style, layout=configs_config_layout)
          config_validation_steps = widgets.IntText(description="validation_steps:", value=load_config.validation_steps, style=configs_config_style, layout=configs_config_layout)
          config_seed = widgets.IntText(description="seed:", value=load_config.seed, style=configs_config_style, layout=configs_config_layout)
          config_mixed_precision = widgets.Text(description="mixed_precision:", value=load_config.mixed_precision, style=configs_config_style, layout=configs_config_layout)
          config_gradient_checkpointing = Dropdown(options=boolean_dropdown, value=load_config.gradient_checkpointing, description="gradient_checkpointing:", style=configs_config_style, layout=configs_config_layout)
          config_enable_xformers_memory_efficient_attention = Dropdown(options=boolean_dropdown, value=load_config.enable_xformers_memory_efficient_attention, description="enable_xformers_memory_efficient_attention:", style=configs_config_style, layout=configs_config_layout)
          # =====================================
          ## Validation_data
          config_subheader2 = widgets.HTML(value="<h3>Validation Data</h3>")
          config_prompts = widgets.Textarea(description="prompts:", value="\n".join(load_config.validation_data.prompts), style=configs_config_style, layout=widgets.Layout(width="500px", height="100px"))
          config_video_length = widgets.IntText(description="video_length:", value=load_config.validation_data.video_length, style=configs_config_style, layout=configs_config_layout)
          config_width = widgets.IntText(description="width:", value=load_config.validation_data.width, style=configs_config_style, layout=configs_config_layout)
          config_height = widgets.IntText(description="height:", value=load_config.validation_data.height, style=configs_config_style, layout=configs_config_layout)
          config_num_inference_steps = widgets.IntText(description="num_inference_steps:", value=load_config.validation_data.num_inference_steps, style=configs_config_style, layout=configs_config_layout)
          config_guidance_scale = widgets.FloatText(description="guidance_scale:", value=load_config.validation_data.guidance_scale, style=configs_config_style, layout=configs_config_layout)
          config_use_inv_latent = Dropdown(options=boolean_dropdown, value=load_config.validation_data.use_inv_latent, description="use_inv_latent:", style=configs_config_style, layout=configs_config_layout)
          config_num_inv_steps = widgets.IntText(description="num_inv_steps:", value=load_config.validation_data.num_inv_steps, style=configs_config_style, layout=configs_config_layout)
          # =====================================
          ## Group widgets
          config_vbox = widgets.VBox([
              config_pretrained_model_path,
              config_output_dir_name,
              config_train_batch_size,
              config_validation_steps,
              config_seed,
              config_mixed_precision,
              config_gradient_checkpointing,
              config_enable_xformers_memory_efficient_attention,
          ])
          config_vbox_validation_data = widgets.VBox([
              config_prompts,
              config_video_length,
              config_width,
              config_height,
              config_num_inference_steps,
              config_guidance_scale,
              config_use_inv_latent,
              config_num_inv_steps
          ])

          # Create listeners
          def save_config(change):
            config = {
                "pretrained_model_path": config_pretrained_model_path.value,
                "output_dir": f"{INFERENCE_OUTPUT_DIR_PATH}/{config_output_dir_name.value}",
                "validation_data": {
                    "prompts": [prompt.strip() for prompt in config_prompts.value.splitlines() if prompt.strip()],
                    "video_length": config_video_length.value,
                    "width": config_width.value,
                    "height": config_height.value,
                    "num_inference_steps": config_num_inference_steps.value,
                    "guidance_scale": config_guidance_scale.value,
                    "use_inv_latent": config_use_inv_latent.value,
                    "num_inv_steps": config_num_inv_steps.value,
                    "dataset_set": load_config.validation_data.dataset_set
                },
                "train_batch_size": config_train_batch_size.value,
                "validation_steps": config_validation_steps.value,
                "resume_from_checkpoint": load_config.resume_from_checkpoint,
                "seed": config_seed.value,
                "mixed_precision": config_mixed_precision.value,
                "gradient_checkpointing": config_gradient_checkpointing.value,
                "enable_xformers_memory_efficient_attention": config_enable_xformers_memory_efficient_attention.value
            }

            # Specify the folder path you want to check
            folder_path = f'{INFERENCE_OUTPUT_DIR_PATH}/{config_output_dir_name.value}'

            # Check if the folder exists
            if os.path.exists(folder_path) and os.path.isdir(folder_path):
              print("\r", f'There is already a folder with the name {config_output_dir_name.value}! Please rename your folder!', end="")

            else:
              # Save updated config back into the yaml file
              with open(sample_yaml_path, "w") as file:
                yaml.dump(config, file, default_style='"', default_flow_style=False, sort_keys=False)

              print("\r", "Saving...", end="")
              time.sleep(2)
              print("\r", "Successfully saved!", end="")

          ## Button Widget and Attach Listener
          config_save_btn = widgets.Button(description="Save", layout=configs_config_button_layout)
          config_save_btn.on_click(save_config)

          # Display fields (same as before)
          display(
            config_subheader1,
            config_vbox,
            config_subheader2,
            config_vbox_validation_data,
            config_save_btn
          )

          # Update the displayed_widgets list
          displayed_widgets.extend([
              config_subheader1,
              config_vbox,
              config_subheader2,
              config_vbox_validation_data,
              config_save_btn
          ])

        else:
          print("The configuration for the yaml is not structured correctly for inference")

# Attach the event handler to the dropdown's 'value' trait
config_files_dropdown.observe(update_load_config, names='value')

# Display the dropdown widget and the output widget
display(config_files_dropdown)

In [ ]:
# @title Run FYP Inference { display-mode: "form" }

# Create layout
layout_single_long_button = widgets.Layout(margin='0px 0px 20px 154px')

# Create widgets
selected_fyp_video_input = widgets.Text(placeholder='Select a video above', description="Video:", disabled=True)
config_file_path = widgets.Text(value='configs/pose_sample.yaml',description="Config File:")
inf_fyp_button = widgets.Button(description="Start Inference", disabled=True, layout=layout_single_long_button)

# Create listeners
## Retrieve video input
def update_fyp_input_video(change):
    selected_video = video_dropdown.value
    selected_fyp_video_input.value = f"{VIDEO_DIR_PATH}/{video_subdir_dropdown.value}/{selected_video}"
    if selected_video:
        inf_fyp_button.disabled = False
    else:
        inf_fyp_button.disabled = True

## Run FYP
def run_fyp_inference(button):
    %cd -q {FYP_DIR_PATH}

    config_file_path_text = config_file_path.value
    video_file_path_text = selected_fyp_video_input.value

    # Specify the folder path you want to check
    load_config = OmegaConf.load(config_file_path_text)
    folder_path = load_config.output_dir

    # Check if the folder exists
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
      print("\r", f'The folder path {load_config.output_dir}! Please rename your folder!', end="")

    else:
      print("\r", "", end="")
      # Start inference
      !TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py \
          --config={config_file_path_text}  \
          --skeleton_path={video_file_path_text}

## Pass on skeleton path
def get_skeleton_path():
    skeleton_path = selected_fyp_video_input.value
    return skeleton_path

## Pass on config path
def get_config_path():
    config_path = config_file_path.value
    return config_path

# Attach Listeners
inf_fyp_button.on_click(run_fyp_inference)
video_fyp_button.on_click(update_fyp_input_video)

# Display fields
display(selected_fyp_video_input)
display(config_file_path)
display(inf_fyp_button)

In [ ]:
#@title Post-Process Inference

# Load the inference config file
sample_yaml_path = f"{CONFIG_DIR_PATH}/pose_sample.yaml"
load_config = OmegaConf.load(sample_yaml_path)

# Creates the skeleton pose gif with the duration matching the sample gifs
postprocess_mmpose(selected_fyp_video_input.value, load_config.validation_data.video_length, load_config.output_dir)

# For every prompt, combine the sample gif with the pose gif and the prompt below
for prompt in load_config.validation_data.prompts:
  postprocess_gif(load_config.output_dir, prompt, load_config.validation_data.width)

In [ ]:
# @title Gif Display
def getInferenceRunFolderContent(folder_name):
  runfolders = []
  for content in os.listdir(INFERENCE_OUTPUT_DIR_PATH):
        content_path = os.path.join(INFERENCE_OUTPUT_DIR_PATH, content)

        # Check if it's a directory and not hidden
        if os.path.isdir(content_path) and not content.startswith("."):
            runfolders.append(content)

  return runfolders

# Init
inference_run_folders = os.listdir(INFERENCE_OUTPUT_DIR_PATH)
inference_run_folder_content = getFolderContent(inference_run_folders)

# Create widgets
gif_output_placeholder = widgets.Output(layout=layout_output)
gif_subdir_dropdown = widgets.Dropdown(options=inference_run_folder_content, description='Folder:', value=None, disabled=False)
gif_dropdown = widgets.Dropdown(options=[], description='Gif:', disabled=True)
gif_display_button = widgets.Button(description="Display", disabled=True, layout=layout_double_button)
gif_refresh_button = widgets.Button(description="Refresh", disabled=False, layout=layout_double_button)

gif_hbox_1 = widgets.HBox([gif_display_button, gif_refresh_button], layout=layout_hbox)

# Create listeners
## Update gif dropdown options based on the selected folder
def gif_subdir_select(change):
    selected_gif_folder = gif_subdir_dropdown.value

    if selected_gif_folder != None:
      selected_GIF_DIR_PATH = f"{INFERENCE_OUTPUT_DIR_PATH}/{selected_gif_folder}/processed"
      selected_gif_dir_content = [file for file in os.listdir(selected_GIF_DIR_PATH) if file.endswith('.gif')]
    else:
      selected_gif_dir_content = []

    gif_dropdown.options = selected_gif_dir_content
    if not selected_gif_dir_content:
        gif_dropdown.disabled = False
        gif_dropdown.value = None
    else:
        gif_dropdown.disabled = False

## Display the selected gif
def display_selected_gif(change):
    selected_gif = gif_dropdown.value
    selected_gif_folder = gif_subdir_dropdown.value

    if selected_gif:
        gif_path = f"{INFERENCE_OUTPUT_DIR_PATH}/{selected_gif_folder}/{selected_gif}"
        gif_display = Image(filename=gif_path, width=512, height=512, embed=True)

        # Clear the output placeholder and display the gif
        with gif_output_placeholder:
            clear_output()
            display(gif_display)

## Refresh folder and directory
def refresh_folder_and_directory(change):
    inference_run_folders = os.listdir(INFERENCE_OUTPUT_DIR_PATH)
    inference_run_folder_content = getFolderContent(inference_run_folders)

    gif_dropdown.options = []
    gif_dropdown.value = None

    gif_subdir_dropdown.options = inference_run_folder_content
    gif_subdir_dropdown.value = None



## Enable button when a valid gif is picked
def enable_button(change):
    if gif_dropdown.value:
        gif_display_button.disabled = False
    else:
        gif_display_button.disabled = True

# Attach Listeners
gif_subdir_dropdown.observe(gif_subdir_select, 'value')
gif_display_button.on_click(display_selected_gif)
gif_refresh_button.on_click(refresh_folder_and_directory)
gif_dropdown.observe(enable_button, 'value')


# Display fields
display(gif_output_placeholder)
display(gif_subdir_dropdown)
display(gif_dropdown)

display(gif_hbox_1)

## Training (WIP)

In [ ]:
# @title Init { display-mode: "form" }

# Charades Data Class from csv
class CharadesData:
  def __init__(self, row):
    id, subject, scene, quality, relevance, verified, script, objects, descriptions, actions, length = row
    self.id = id
    self.subject = subject
    self.scene = scene
    self.quality = quality
    self.relevance = relevance
    self.verified = verified
    self.script = script
    self.objects = objects.split(";")
    self.descriptions = descriptions
    self.length = length
    self.actions = {}

    # Convert actions in proper data structure ("class_id time_start time_end" -> class_id: [time_start, time_end])
    if len(actions) != 0:
      action_substrings = actions.split(';')
      for substring in action_substrings:
        parts = substring.split()
        key = parts[0]
        values = [self.convert_to_ms(parts[1]), self.convert_to_ms(parts[2])]
        self.actions[key] = values

  # For printing
  def __str__(self):
        return f"ID: {self.id}, Subject: {self.subject}, Scene: {self.scene}, Quality: {self.quality}, Relevance: {self.relevance}, Verified: {self.verified}, Script: {self.script}, Objects: {self.objects}, Descriptions: {self.descriptions}, Actions: {self.actions}, Length: {self.length}"

  # Helper function to convert time into ms
  def convert_to_ms(self, seconds):
    ss,ms = seconds.split('.')
    total_ms = 1000*int(ss) + int(ms)
    return total_ms

  # Caption getter with template
  def getCaption(self, index):
    return f"In a {self.scene} setting, within the context of '{self.script}', the action '{action_descriptions[list(self.actions.keys())[index]]}' is taking place."


action_descriptions = {}
charades_all = []

# Load classes lookup table
with open('/content/charades_lookup/Charades_v1_classes.txt', 'r') as file:
    for line in file:
        code, description = line.strip().split(' ', 1)
        action_descriptions[code] = description

# Load charades data A
with open('/content/charades_lookup/Charades_v1_train.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader, None)

    for row in csv_reader:
        charadeData = CharadesData(row)
        charades_all.append(charadeData)

# Load charades data B
with open('/content/charades_lookup/Charades_v1_test.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader, None)

    for row in csv_reader:
        charadeData = CharadesData(row)
        charades_all.append(charadeData)

clear_output()
print("Data load successful!")

In [ ]:
# @title Dataset Preload (Video) { display-mode: "form" }

# Init
dataset_dir_folders = os.listdir(VIDEO_DIR_PATH)

## Training env
training_dataset = None
training_idx = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
training_branch = f"{TRAINING_CONTENT_DIR_PATH}/{training_idx}"

# Create layout
config_style = {'description_width': '100px'}
config_layout = widgets.Layout(width="300px")
config_button_layout = widgets.Layout(margin='0px 0px 20px 154px')

# Create widgets
dataset_folder_dropdown = widgets.Dropdown(options=[dir for dir in dataset_dir_folders if not dir.startswith(".")], description='Dataset Folder:', value=None, layout=config_layout, style=config_style)
dataset_cutting_button = widgets.Button(description="Start Cutting", disabled=True, layout=config_button_layout)

# Create listeners
def dataset_dir_select(change):
  if dataset_folder_dropdown:
    dataset_folder_content = os.listdir(f"{VIDEO_DIR_PATH}/{dataset_folder_dropdown.value}")
    total_dataset_videos = len([file for file in dataset_folder_content if file.endswith('.mp4')])

    dataset_cutting_button.disabled = False
    dataset_cutting_button.description = f"Start Cutting ({total_dataset_videos})"

def video_cutting_select(change):
  os.makedirs(training_branch, exist_ok=True)
  training_dataset = f"{VIDEO_DIR_PATH}/{dataset_folder_dropdown.value}"

  # Loop video files from selected dataset folder
  for video_file in os.listdir(training_dataset):
    video, ext = os.path.splitext(video_file)

    # Ignore non video files (Eg: .ipynb_checkpoint and csv)
    if ext != ".mp4":
      continue

    video_folder = f"{training_branch}/{video}"
    if not os.path.exists(video_folder):
      os.mkdir(video_folder)

      # Retrieve charade object by ID
      charade_data = None
      for charade in charades_all:
        if charade.id == video:
            charade_data = charade
            break

      # If no clipping required, keep whole video
      if not charade_data.actions:
        print(f"No clipping needed")
      else:
        print(f"Clipping {video}")
        charade_actions = charade_data.actions.items()
        total_charade_actions = len(charade_actions)
        for i, (class_id, timings) in enumerate(charade_actions):

          input_video = f"{training_dataset}/{video_file}"
          output_video = f"{video_folder}/{video}{i+1:02}{ext}"

          print(f"#{i+1}/{total_charade_actions}: {timings[0]}ms to {timings[0]+timings[1]}ms [I:{input_video}] [O:{output_video}]")
          !ffmpeg -i {input_video} -ss {timings[0]}ms -t {timings[1]}ms -c:v libx264 -c:a aac {output_video} -loglevel quiet
    else:
      print(f"Folder already exist for video_id: {video}. Skipping ...")
  print("Finished Clipping")

# Attach Listeners
dataset_folder_dropdown.observe(dataset_dir_select, 'value')
dataset_cutting_button.on_click(video_cutting_select)

# Display fields
display(dataset_folder_dropdown)
display(dataset_cutting_button)

In [ ]:
# @title Dataset Preload (Metadata) { display-mode: "form" }

training_metadata_file = f"{training_branch}/metadata.tsv"
with open(training_metadata_file, 'w', newline='', encoding='utf-8') as tsvfile:
  fieldnames = ['part_id', 'clip_id', 'caption']
  writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
  writer.writeheader()

  for part_id in os.listdir(training_branch):
    folder_path = os.path.join(training_branch, part_id)

    # Ignore non video files (Eg: .ipynb_checkpoint and csv)
    if not os.path.isdir(folder_path) or part_id.startswith("."):
      continue

    charade_data = None
    for charade in charades_all:
      if charade.id == part_id:
          charade_data = charade
          break

    if not charade_data:
      print("Missing charades data, skipping ...")
      continue

    # Sort by video sub-id to maintain order
    training_video_files = sorted(os.listdir(folder_path), key=lambda x: int(os.path.splitext(x)[0][-2:]))
    for i, clip in enumerate(training_video_files):
      caption = charade_data.getCaption(i)
      writer.writerow({
          'part_id': part_id,
          'clip_id': clip,
          'caption': caption
      })
print(f"TSV created: {training_metadata_file}")

In [ ]:
# @title Training Configuration { display-mode: "form" }

# Init
%cd -q {FYP_DIR_PATH}

# Initialize load_config with a default value
load_config = None

# Get a list of all files in the directory
config_files = [f for f in os.listdir(CONFIG_DIR_PATH) if os.path.isfile(os.path.join(CONFIG_DIR_PATH, f))]

# Define a container for displayed widgets
displayed_widgets = []

# Create a dropdown widget with the list of config files
config_files_dropdown = Dropdown(
  options=["- Select an Item -"] + config_files,
  description='Select a Config File:',
  layout=Layout(width="500px"),
  style={'description_width': '150px'}
)

# Function to clear displayed widgets (excluding the dropdown)
def clear_displayed_widgets():
    for widget in displayed_widgets:
        widget.close()
    displayed_widgets.clear()
    display(config_files_dropdown)  # Display the dropdown again

# Function to update the load_config variable based on the selected filename
def update_load_config(change):
    global load_config
    selected_filename = change.new
    if selected_filename and selected_filename != "- Select an Item -":
        clear_output(wait=True)  # Clear the output area
        clear_displayed_widgets()  # Clear previously displayed widgets

        # Load yaml file
        sample_yaml_path = f"{CONFIG_DIR_PATH}/{selected_filename}"
        # load_config = OmegaConf.load(sample_yaml_path)
        with open(sample_yaml_path, 'r') as yaml_file:
          load_config = yaml.load(yaml_file, Loader=yaml.FullLoader)
        print(f"Editing config from: {sample_yaml_path}")

        # Check if the yaml configuration matches expected training config
        if compare_dict_structure(expected_training_config, load_config):

          # Create a list of model names and paths in the directory
          model_options = create_model_list()
          video_options = create_video_path_list()
          video_path_default = load_config["train_data"]["video_path"]

          if not (os.path.exists(video_path_default) and os.path.isdir(video_path_default)):
            video_path_default = "/content/training_content/"
          # =====================================
          ## Basic Data
          config_subheader1 = widgets.HTML(value="<h3>Basic Data</h3>")
          config_pretrained_model_path = Dropdown(options=model_options, description="pretrained_model_path:", value=load_config["pretrained_model_path"], style=configs_config_style, layout=configs_config_layout)
          config_output_dir_name = widgets.Text(description="output_dir_name:", value="", style=configs_config_style, layout=configs_config_layout)
          config_learning_rate = widgets.FloatText(description="learning_rate:", value=load_config["learning_rate"], style=configs_config_style, layout=configs_config_layout)
          # =====================================
          # train_data
          config_subheader2 = widgets.HTML(value="<h3>Train Data</h3>")
          config_video_path = Dropdown(options=video_options, description="video_path:", value=video_path_default, style=configs_config_style, layout=configs_config_layout)
          config_n_sample_frames = widgets.IntText(description="n_sample_frames:", value=load_config["train_data"]["n_sample_frames"], style=configs_config_style, layout=configs_config_layout)
          config_train_data_width = widgets.IntText(description="width:", value=load_config["train_data"]["width"], style=configs_config_style, layout=configs_config_layout)
          config_sample_frame_rate = widgets.IntText(description="sample_frame_rate:", value=load_config["train_data"]["sample_frame_rate"], style=configs_config_style, layout=configs_config_layout)
          # =====================================
          config_train_batch_size = widgets.IntText(description="train_batch_size:", value=load_config["train_batch_size"], style=configs_config_style, layout=configs_config_layout)
          config_max_train_steps = widgets.IntText(description="max_train_steps:", value=load_config["max_train_steps"], style=configs_config_style, layout=configs_config_layout)
          config_seed = widgets.IntText(description="seed:", value=load_config["seed"], style=configs_config_style, layout=configs_config_layout)
          config_mixed_precision = widgets.Text(description="mixed_precision:", value=load_config["mixed_precision"], style=configs_config_style, layout=configs_config_layout)
          config_use_8bit_adam = Dropdown(options=boolean_dropdown, value=load_config["use_8bit_adam"], description="config_use_8bit_adam:", style=configs_config_style, layout=configs_config_layout)
          config_gradient_checkpointing = Dropdown(options=boolean_dropdown, value=load_config["gradient_checkpointing"], description="gradient_checkpointing:", style=configs_config_style, layout=configs_config_layout)
          config_enable_xformers_memory_efficient_attention = Dropdown(options=boolean_dropdown, value=load_config["enable_xformers_memory_efficient_attention"], description="enable_xformers_memory_efficient_attention:", style=configs_config_style, layout=configs_config_layout)
          # =====================================
          ## Button Widget
          config_save_btn = widgets.Button(description="Save", layout=configs_config_button_layout)
          ## Group widgets
          config_vbox = widgets.VBox([
              config_pretrained_model_path,
              config_output_dir_name,
              config_learning_rate,
              config_train_batch_size,
              config_max_train_steps,
              config_seed,
              config_mixed_precision,
              config_use_8bit_adam,
              config_gradient_checkpointing,
              config_enable_xformers_memory_efficient_attention
          ])
          config_vbox_train_data = widgets.VBox([
              config_video_path,
              config_n_sample_frames,
              config_train_data_width,
              config_sample_frame_rate,
          ])

          # Display fields (same as before)
          display(
            config_subheader1,
            config_vbox,
            config_subheader2,
            config_vbox_train_data,
            config_save_btn
          )

          # Create listeners
          def save_config(change):
            config = {
                "pretrained_model_path": config_pretrained_model_path.value,
                "output_dir": f"{CUSTOM_MODEL_DIR_PATH}/{config_output_dir_name.value}",
                "train_data": {
                    "video_path": config_video_path.value,
                    "n_sample_frames": config_n_sample_frames.value,
                    "width": config_train_data_width.value,
                    "sample_frame_rate": config_sample_frame_rate.value
                },
                "learning_rate": config_learning_rate.value,
                "train_batch_size": config_train_batch_size.value,
                "max_train_steps": config_max_train_steps.value,
                "trainable_modules": load_config["trainable_modules"],
                "seed": config_seed.value,
                "mixed_precision": config_mixed_precision.value,
                "use_8bit_adam": config_use_8bit_adam.value,
                "gradient_checkpointing": config_gradient_checkpointing.value,
                "enable_xformers_memory_efficient_attention": config_enable_xformers_memory_efficient_attention.value
            }

            if config_video_path.value == f"{TRAINING_CONTENT_DIR_PATH}/":
              print("\r", "Please choose a training folder!", end="")

            else:
              #Save updated config back into yaml file
              with open(sample_yaml_path, "w") as file:
                yaml.dump(config, file, default_style='"', default_flow_style=False, sort_keys=False)

              print("\r", "Saving...", end="")
              time.sleep(2)
              print("\r", "Successfully saved!", end="")

          # Attach Listeners
          config_save_btn.on_click(save_config)

          # Update the displayed_widgets list
          displayed_widgets.extend([
              config_subheader1,
              config_vbox,
              config_subheader2,
              config_vbox_train_data,
              config_save_btn,
          ])

        else:
          print("The configuration for the yaml is not structured correctly for training")

# Attach the event handler to the dropdown's 'value' trait
config_files_dropdown.observe(update_load_config, names='value')

# Display the dropdown widgets
display(config_files_dropdown)


In [ ]:
# @title Run Training { display-mode: "form" }

%cd -q {FYP_DIR_PATH}

output_label = widgets.Label(value="Output will appear here:")
display(output_label)

def run_command_and_display_output(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True, shell=True)
    for line in process.stdout:
        output_label.value = line.strip()  # Update the label with the live output
    process.wait()

run_command_and_display_output('TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch train_followyourpose.py --config="configs/pose_train.yaml"')